# 赛题分析
论文的增量消歧(Continuous Name Disambiguation)
任务描述：线上系统每天会新增大量的论文，如何准确快速的将论文分配到系统中已有作者档案，这是线上学术系统最亟待解决的问题。所以问题抽象定义为：给定一批新增论文以及系统已有的作者论文集，最终目的是把新增论文分配到正确的作者档案中。
参考方法：增量消歧任务与冷启动消歧的任务不同，它是基于有一定作者档案的基础，对新增论文进行分配。所以，容易直接想到的方法就是将已有的作者档案与新增论文进行比较，提取合作者，单位机构或者会议期刊之间相似度的传统特征，随后利用svm之类的传统分类器进行分类。

# 数据处理

## 生成训练数据

In [1]:
import json
import random
from pyjarowinkler import distance
import numpy as np
from tqdm.notebook import tqdm
import torch
import torch.utils.data as data
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F

In [2]:
# 训练集中的作者论文信息
with open("D:/Zhang/Datasets/论文增量消歧/train/train_author.json", "r") as f2:
    author_data = json.load(f2)

# 训练集的论文元信息
with open("D:/Zhang/Datasets/论文增量消歧/train/train_pub.json", "r") as f2:
    pubs_dict = json.load(f2)

print(len(author_data))


name_train = set()

# 筛选训练集，只取同名作者数大于等于5个的名字作为训练集。
for name in author_data:
    persons = author_data[name]
    if(len(persons) > 5):
        name_train.add((name))

print(len(name_train))

221
196


In [3]:
# 采样500个训练例子，一个训练例子包含paper和正例作者以及5个负例作者（正负例比=1：5）

# 记录paper所属作者和名字
paper2aid2name = {}

for author_name in name_train:
    persons = author_data[author_name]
    for person in persons:
        paper_list = persons[person]
        for paper_id in paper_list:
            paper2aid2name[paper_id] = (author_name, person)

print(len(paper2aid2name))
# print(paper2aid2name)

total_paper_list = list(paper2aid2name.keys())

198607


In [4]:
# 采样10000篇paper作为训练集
train_paper_list = random.sample(total_paper_list, 10000)

# 把采样的500篇paper转变成对应的训练例子，一个训练例子包含paper和正例作者以及5个负例作者（正负例比=1：5）
train_instances = []
for paper_id in train_paper_list:
    
    # 保存对应的正负例
    pos_ins = set()
    neg_ins = set()
    
    paper_author_name = paper2aid2name[paper_id][0]
    paper_author_id = paper2aid2name[paper_id][1]
    
    pos_ins.add((paper_id, paper_author_id))
    
    # 获取同名的所有作者(除了本身)作为负例的candidate
    persons = list(author_data[paper_author_name].keys())
    persons.remove(paper_author_id)
    assert len(persons) == (len(list(author_data[paper_author_name].keys())) - 1)
    
    # 每个正例采样5个负例
    neg_author_list = random.sample(persons, 1)
    for i in neg_author_list:
        neg_ins.add((paper_id, i))
        
    train_instances.append((pos_ins, neg_ins))
    
print(len(train_instances))  

10000


In [5]:
train_instances[0]

({('x34a5vCM', 'WFMrcMVu')}, {('x34a5vCM', 'Np6W5EhI')})

## 生成特征
在这里，我们只提取paper与author之间的coauthor相关的特征:

In [6]:
# 作者所发表过论文的keyword信息
aid2keywords = dict()
for name in author_data:
    for author_id in author_data[name]:
        aid2keywords[author_id] = set()
        paper_list = author_data[name][author_id]
        for paper in paper_list:
            if 'keywords' not in pubs_dict[paper]:
                continue
            for keyword in pubs_dict[paper]['keywords']:
                aid2keywords[author_id].add(keyword)

In [7]:
# 这里定义了俩个生成特征所需的函数
from pyjarowinkler import distance


# 对author_name 进行清洗
def clean_name(name):
    if name is None:
        return ""
    x = [k.strip() for k in name.lower().strip().replace(".", "").replace("-", " ").replace("_", ' ').split()]
    # x = [k.strip() for k in name.lower().strip().replace("-", "").replace("_", ' ').split()]
    full_name = ' '.join(x)
    name_part = full_name.split()
    if(len(name_part) >= 1):
        return full_name
    else:
        return None

# 找出paper中author_name所对应的位置
def delete_main_name(author_list, name):
    score_list = []
    name = clean_name(name)
    author_list_lower = []
    for author in author_list:
        author_list_lower.append(author.lower())
    name_split = name.split()
    for author in author_list_lower:
        # lower_name = author.lower()
        score = distance.get_jaro_distance(name, author, winkler=True, scaling=0.1)
        author_split = author.split()
        inter = set(name_split) & set(author_split)
        alls = set(name_split) | set(author_split)
        score += round(len(inter)/len(alls), 6)
        score_list.append(score)

    rank = np.argsort(-np.array(score_list))
    return_list = [author_list_lower[i] for i in rank[1:]]

    return return_list, rank[0]

# 训练集特征生成函数
def process_feature(pos_ins, paper_coauthors, paper_keywords):
    # pos_ins: (paper_id, author_id)
    # paper_coauthors: [author_name1, author_name2...]
    feature_list = []

    paper = pos_ins[0] 
    author = pos_ins[1]
    paper_author_id = author
    paper_name = paper2aid2name[paper][0]
    
    # 从作者的论文列表中把该篇论文去掉，防止训练出现bias
    doc_list = []
    for doc in author_data[paper_name][author]:
        if(doc != paper):
            doc_list.append(doc)
    for doc in doc_list:
        if doc == paper:
            print("error!")
            exit()
    
    # 保存作者的所有paper的coauthors以及各自出现的次数(作者所拥有论文的coauthors)
    candidate_authors_int = defaultdict(int)

    total_author_count = 0
    for doc in doc_list:
        
        doc_dict = pubs_dict[doc]
        author_list = []

        paper_authors = doc_dict['authors']
        paper_authors_len = len(paper_authors)
        paper_authors = random.sample(paper_authors, min(50, paper_authors_len))
    
        for author in paper_authors:                
            clean_author = clean_name(author['name'])
            if(clean_author != None):
                author_list.append(clean_author)
        if(len(author_list) > 0):
            # 获取paper中main author_name所对应的位置
            _, author_index = delete_main_name(author_list, paper_name)

            # 获取除了main author_name外的coauthor
            for index in range(len(author_list)):
                if(index == author_index):
                    continue
                else:
                    candidate_authors_int[author_list[index]] += 1
                    total_author_count += 1

    # author 的所有不同coauthor name
    author_keys = list(candidate_authors_int.keys())

    if ((len(author_keys) == 0) or (len(paper_coauthors) == 0)):
        feature_list.extend([0.] * 6)
    else:
        co_coauthors = set(paper_coauthors) & set(author_keys)
        coauthor_len = len(co_coauthors)
        
        co_coauthors_ratio_for_paper = round(coauthor_len / len(paper_coauthors), 6)
        co_coauthors_ratio_for_author = round(coauthor_len / len(author_keys), 6)
        
        coauthor_count = 0
        for coauthor_name in co_coauthors:
            coauthor_count += candidate_authors_int[coauthor_name]
            
        co_coauthors_ratio_for_author_count = round(coauthor_count / total_author_count, 6)

        co_keywords_count = len(paper_keywords & aid2keywords[paper_author_id])/len(aid2keywords)
        
        # 计算了5维paper与author所有的paper的coauthor相关的特征：
        #    1. 不重复的coauthor个数
        #    2. 不重复的coauthor个数 / paper的所有coauthor的个数
        #    3. 不重复的coauthor个数 / author的所有paper不重复coauthor的个数
        #    4. coauthor个数（含重复）
        #    5. coauthor个数（含重复）/ author的所有paper的coauthor的个数（含重复）
        #    6. cokeyword个数
        feature_list.extend([coauthor_len, co_coauthors_ratio_for_paper, co_coauthors_ratio_for_author, coauthor_count, co_coauthors_ratio_for_author_count, co_keywords_count])
        
#         print(feature_list)
    return feature_list

In [8]:
#生成所有正例以及负例的特征
from collections import defaultdict

pos_features = []
neg_features = []

print(len(train_instances))

for ins in tqdm(train_instances):
    
    pos_set = ins[0]
    neg_set = ins[1]
    paper_id = list(pos_set)[0][0]
    paper_name = paper2aid2name[paper_id][0]
    
    
    author_list = []
    # 获取paper的coauthors
    paper_coauthors = []
    paper_keywords = set()
    if 'keywords' in pubs_dict[paper_id]:
        paper_keywords.update(pubs_dict[paper_id]['keywords'])

    paper_authors = pubs_dict[paper_id]['authors']
    paper_authors_len = len(paper_authors)
    # 只取前50个author以保证效率
    paper_authors = random.sample(paper_authors, min(50, paper_authors_len))

    for author in paper_authors:                
        clean_author = clean_name(author['name'])
        if(clean_author != None):
            author_list.append(clean_author)
    if(len(author_list) > 0):
        # 获取paper中main author_name所对应的位置
        _, author_index = delete_main_name(author_list, paper_name)
        
        # 获取除了main author_name外的coauthor
        for index in range(len(author_list)):
            if(index == author_index):
                continue
            else:
                paper_coauthors.append(author_list[index])
        
    
        for pos_ins in pos_set:
            pos_features.append(process_feature(pos_ins, paper_coauthors, paper_keywords))
    
        for neg_ins in neg_set:
            neg_features.append(process_feature(neg_ins, paper_coauthors, paper_keywords))
            
print(np.array(pos_features).shape)
print(np.array(neg_features).shape)    

10000



(9998, 6)
(9998, 6)


In [9]:
# 构建正负例
train_ins = []
for ins in tqdm(pos_features):
    train_ins.append((ins, 1))

for ins in neg_features:
    train_ins.append((ins, 0))

print(np.array(train_ins).shape)

random.shuffle(train_ins)

x_train= []
y_train = [] 
for ins in tqdm(train_ins):
    x_train.append(ins[0])
    y_train.append(ins[1])


(19996, 2)


# 加载处理测试数据

In [35]:
# 训练集中的作者论文信息
with open("D:/Zhang/Datasets/论文增量消歧/cna_data/whole_author_profile.json", "r") as f2:
    test_author_data = json.load(f2)

# 训练集的论文元信息
with open("D:/Zhang/Datasets/论文增量消歧/cna_data/whole_author_profile_pub.json", "r") as f2:
    test_pubs_dict = json.load(f2)

# 待分配论文集
with open("D:/Zhang/Datasets/论文增量消歧/cna_data/cna_valid_unass_competition.json", "r") as f2:
    unass_papers = json.load(f2)

with open("D:/Zhang/Datasets/论文增量消歧/cna_data/cna_valid_pub.json", "r") as f2:
    unass_papers_dict = json.load(f2)

# with open("cna_data/new_test_author_data.json", 'r') as files:
#     new_test_author_data = json.load(files)
# 简单处理whole_author_profile，将同名的作者合并：
# 为了效率，预处理new_test_author_data中的paper，将其全部处理成paper_id + '-' + author_index的形式。
new_test_author_data = {}
for author_id, author_info in test_author_data.items():
    author_name = author_info['name']
    author_papers = author_info['papers']
    newly_papers = []

    for paper_id in author_papers:

        paper_authors = test_pubs_dict[paper_id]['authors']
        paper_authors_len = len(paper_authors)
        
        # 只利用author数小于50的paper，以保证效率
        if(paper_authors_len > 50):
            continue
#         paper_authors = random.sample(paper_authors, min(50, paper_authors_len))
        author_list = []
        for author in paper_authors:                
            clean_author = clean_name(author['name'])
            if(clean_author != None):
                author_list.append(clean_author)
        if(len(author_list) > 0):
            # 获取paper中main author_name所对应的位置
            _, author_index = delete_main_name(author_list, paper_name)

            new_paper_id = str(paper_id) + '-' + str(author_index)
            newly_papers.append(new_paper_id)
        
        
    if(new_test_author_data.get(author_name) != None):
        new_test_author_data[author_name][author_id] = newly_papers
    else:
        tmp = {}
        tmp[author_id] = newly_papers
        new_test_author_data[author_name] = tmp
print(len(new_test_author_data))

320


In [36]:
# test集的特征生成函数，与train类似
def process_test_feature(pair, new_test_author_data, test_pubs_dict, paper_coauthors, paper_keywords):
    
    feature_list = []

    paper = pair[0] 
    author = pair[1]
    paper_name = pair[2]
    
    doc_list = new_test_author_data[paper_name][author]

    
    # 保存作者的所有coauthors以及各自出现的次数(作者所拥有论文的coauthors)
    candidate_authors_int = defaultdict(int)

    total_author_count = 0
    for doc in doc_list:
        doc_id = doc.split('-')[0]
        author_index = doc.split('-')[1]
        doc_dict = test_pubs_dict[doc_id]
        author_list = []

        paper_authors = doc_dict['authors']
        paper_authors_len = len(paper_authors)
        paper_authors = random.sample(paper_authors, min(50, paper_authors_len))
    
        for author in paper_authors:                
            clean_author = clean_name(author['name'])
            if(clean_author != None):
                author_list.append(clean_author)
        if(len(author_list) > 0):

            # 获取除了main author_name外的coauthor
            for index in range(len(author_list)):
                if(index == author_index):
                    continue
                else:
                    candidate_authors_int[author_list[index]] += 1
                    total_author_count += 1

    author_keys = list(candidate_authors_int.keys())

    if ((len(author_keys) == 0) or (len(paper_coauthors) == 0)):
        feature_list.extend([0.] * 6)
    else:
        co_coauthors = set(paper_coauthors) & set(author_keys)
        coauthor_len = len(co_coauthors)
        
        co_coauthors_ratio_for_paper = round(coauthor_len / len(paper_coauthors), 6)
        co_coauthors_ratio_for_author = round(coauthor_len / len(author_keys), 6)
        
        coauthor_count = 0
        for coauthor_name in co_coauthors:
            coauthor_count += candidate_authors_int[coauthor_name]
            
        
        
        co_coauthors_ratio_for_author_count = round(coauthor_count / total_author_count, 6)
        
        co_keywords_count = len(set(paper_keywords) & aid2keywords[paper_author_id])

        # 计算了5维paper与author所有的paper的coauthor相关的特征：
        #    1. 不重复的coauthor个数
        #    2. 不重复的coauthor个数 / paper的所有coauthor的个数
        #    3. 不重复的coauthor个数 / author的所有paper不重复coauthor的个数
        #    4. coauthor个数（含重复）
        #    4. coauthor个数（含重复）/ author的所有paper的coauthor的个数（含重复）
        feature_list.extend([coauthor_len, co_coauthors_ratio_for_paper, co_coauthors_ratio_for_author, coauthor_count, co_coauthors_ratio_for_author_count, co_keywords_count])
        
#         print(feature_list)
    return feature_list

In [37]:
print(len(unass_papers))

count = 0

# 存储paper的所有candidate author id
paper2candidates = defaultdict(list)
# 存储对应的paper与candidate author的生成特征
paper2features = defaultdict(list)

for u_p in unass_papers:
    paper_id = u_p.split('-')[0]
    author_index = int(u_p.split('-')[1])
    author_list = []
    
    # 获取paper的coauthors
    paper_coauthors = []
    paper_name = ''
    paper_authors = unass_papers_dict[paper_id]['authors']
    paper_keywords = unass_papers_dict[paper_id]['keywords']
#     paper_authors_len = len(paper_authors)
#     paper_authors = random.sample(paper_authors, min(50, paper_authors_len))

    for author in paper_authors:                
        clean_author = clean_name(author['name'])
        if(clean_author != None):
            author_list.append(clean_author)
    if(len(author_list) > 0):
        
        # 获取除了main author_name外的coauthor
        for index in range(len(author_list)):
            if(index == author_index):
                continue
            else:
                paper_coauthors.append(author_list[index])
    
    # 简单使用精确匹配找出candidate_author_list
    paper_name = '_'.join(clean_name(paper_authors[author_index]['name']).split())
    if(new_test_author_data.get(paper_name) != None):
        candidate_author_list = new_test_author_data[paper_name]
        for candidate_author in candidate_author_list:
            pair = (paper_id, candidate_author, paper_name)
            paper2candidates[paper_id].append(candidate_author)
            paper2features[paper_id].append(process_test_feature(pair, new_test_author_data, test_pubs_dict, paper_coauthors, paper_keywords))
        count += 1
print(count)
assert len(paper2candidates) == len(paper2features)
print(len(paper2candidates))

9170
8792
8761


# MLP

In [10]:
class trainData(data.Dataset):
    def __init__(self, instances):
        self.instances = instances
        
    def __getitem__(self, index):
        feature, label = self.instances[index]
        return torch.FloatTensor(feature).cuda(), torch.FloatTensor([label]).cuda()
    
    def __len__(self):
        return len(self.instances)
    
class testData(data.Dataset):
    def __init__(self, instances):
        self.instances = instances
        
    def __getitem__(self, index):
        feature = self.instances[index]
        return torch.FloatTensor(feature).cuda()
    
    def __len__(self):
        return len(self.instances)

In [52]:
split = int(len(train_ins)/5)
train_data, test_data = train_ins[split:], train_ins[:split]
print(len(train_data), len(test_data))

train_data = trainData(train_data)
train_loader = data.DataLoader(train_data, batch_size=4, shuffle=True)
test_data = trainData(test_data)
test_loader = data.DataLoader(test_data, batch_size=4)

15997 3999


In [53]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2):
        super().__init__()
        self.input_size = input_size
        self.layer1 = nn.Linear(input_size, hidden_size1)
        self.layer2 = nn.Linear(hidden_size1, hidden_size2)
        self.layer3 = nn.Linear(hidden_size2, 1)
        
    def forward(self, x):
        x = self.layer1(x)
        x = F.relu(x)
        x = self.layer2(x)
        x = F.relu(x)
        x = self.layer3(x)
        x = torch.sigmoid(x)
        return x

In [58]:
net = MLP(6,10,10)
net = net.cuda()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

In [56]:
a = torch.Tensor([1,2,3,3.0])
3 in a

True

In [59]:
net.train()
for epoch in range(1):
    loss_epoch = []
    for feature, label in tqdm(train_loader):
        pred = net(feature)
        if 0 in pred or 1 in pred:
            print(pred)
        loss = (-label*torch.log(pred)-(1-label)*torch.log(1-pred)).sum()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_epoch.append(loss.item())
    print(f"epoch {epoch}: {sum(loss_epoch)/len(loss_epoch)}")

tensor([[0.9999],
        [0.6509],
        [0.5895],
        [1.0000]], device='cuda:0', grad_fn=<SigmoidBackward>)

epoch 0: nan


In [42]:
net.eval()
result = []
for feature, label in tqdm(test_loader):
    pred = net(feature)
    for idx in range(pred.shape[0]):
        if (pred[idx].item() >= 0.5 and label[idx].item()==1) or (pred[idx].item()<0.5 and label[idx].item()==0):
            result.append(1)
        else:
            result.append(0)
acc = sum(result)/len(result)
print(f"accuracy: {acc}")


accuracy: 0.5011252813203301


# 利用svm进行训练

In [33]:
from sklearn.svm import SVC
from sklearn.externals import joblib

clf = SVC(probability=True)
clf.fit(x_train, y_train)

D:\ProgramData\Anaconda3\envs\biendata\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
D:\ProgramData\Anaconda3\envs\biendata\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=True, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

# 利用训练好的svm模型去预测

In [34]:
result_dict = defaultdict(list)
for paper_id, ins_feature_list in paper2features.items(): 
    score_list = []
    for ins in ins_feature_list:
        # 利用svm对一篇paper的所有candidate author去打分，利用分数进行排序，取top-1 author作为预测的author
        prob_pred = clf.predict_proba([ins])[:, 1]
        score_list.append(prob_pred[0])
    rank = np.argsort(-np.array(score_list))
    #取top-1 author作为预测的author
    predict_author = paper2candidates[paper_id][rank[0]]
    result_dict[predict_author].append(paper_id)

with open("D:/Zhang/Datasets/论文增量消歧/cna_data/result.json", 'w') as files:
    json.dump(result_dict, files, indent = 4)

NameError: name 'paper2features' is not defined

### 提交评测后，结果为F1 = 0.63100

In [ ]:
for idx, x in enumerate(x_train):
    print(x, y_train[idx])

## 决策树

In [ ]:
from sklearn.ensemble import  AdaBoostClassifier
 
clf = AdaBoostClassifier()
clf.fit(x_train, y_train)

In [ ]:
result_dict = defaultdict(list)
for paper_id, ins_feature_list in paper2features.items(): 
    score_list = []
    for ins in ins_feature_list:
        # 利用svm对一篇paper的所有candidate author去打分，利用分数进行排序，取top-1 author作为预测的author
        prob_pred = clf_decision_tree.predict_proba([ins])[:, 1]
        score_list.append(prob_pred[0])
    rank = np.argsort(-np.array(score_list))
    #取top-1 author作为预测的author
    predict_author = paper2candidates[paper_id][rank[0]]
    result_dict[predict_author].append(paper_id)

with open("D:/Zhang/Datasets/论文增量消歧/cna_data/result.json", 'w') as files:
    json.dump(result_dict, files, indent = 4)